# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

c:\Users\mybha\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob
import pandas as pd
# Write your code below.
pricedata = os.getenv("PRICE_DATA")
pfiles = glob(os.path.join(pricedata, "**/*.parquet"), recursive = True)
pfiles

['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [14]:
# Write your code below.
dd_px = dd.read_parquet(pfiles).set_index("Ticker")
dd_px
    
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
     lambda x: x.assign(
         Close_lag_1 = x['Close'].shift(1),
         #Adj_Close_lag_1 = x['Adj_Close'].shift(1),
         hi_lo_range=x['High'] - x['Low']
    ))
dd_feat = dd_shift.assign(Returns = lambda x: x['Close']/x['Close_lag_1'] - 1)

C:\Users\mybha\AppData\Local\Temp\ipykernel_10952\507234544.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [17]:
# Write your code below.
dd_feat_pd = dd_feat.compute()
dd_feat_pd
dd_feat_pd['Returns_ma_10'] = dd_feat_pd['Returns'].rolling(window=10).mean()
print(dd_feat_pd.head())

             Date     Close      High       Low      Open       Volume  Year  \
Ticker                                                                         
AAPL   2000-01-03  0.843076  0.847313  0.765877  0.789884  535796800.0  2000   
AAPL   2000-01-04  0.771997  0.833191  0.762111  0.815303  512377600.0  2000   
AAPL   2000-01-05  0.783294  0.832720  0.775762  0.781411  778321600.0  2000   
AAPL   2000-01-06  0.715508  0.805889  0.715508  0.799299  767972800.0  2000   
AAPL   2000-01-07  0.749401  0.760699  0.719275  0.726806  460734400.0  2000   

        Close_lag_1  hi_lo_range   Returns  Returns_ma_10  
Ticker                                                     
AAPL            NaN     0.081436       NaN            NaN  
AAPL       0.843076     0.071080 -0.084310            NaN  
AAPL       0.771997     0.056958  0.014633            NaN  
AAPL       0.783294     0.090380 -0.086539            NaN  
AAPL       0.715508     0.041424  0.047369            NaN  


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

With Dusk also, we can compute mean on Returns;

Dusk is lazy in evaluation, performs parallel processing behind the scene for faster execution, hence for larger datasets it's preferable compared to panda.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.